In [ ]:
import sys
sys.path.append('../')

from src.azure_cosmos_db import AzureCosmos
from src.config import COSMOS_ENDPOINT, COSMOS_KEY, COSMOS_DATABASE_NAME

import openai
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import ast
import re
import uuid
import json
import time
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

openai.api_key  = ''
openai.api_type = 'azure'
openai.api_base = ''
openai.api_version = ''

In [ ]:
def get_context(query):
    openai.api_key  = '9fcf6b05a5a44a4ebbf4daa0979eae1e'
    openai.api_type = 'azure'
    openai.api_base = 'https://openai-ct.openai.azure.com/'
    openai.api_version = '2023-03-15-preview'
    
    start=time.time()
    query_vector=openai.Embedding.create(input=query,engine='text-embedding-ada-002')['data'][0]['embedding']
    end=time.time()
    print(end-start,'s')
    results=search_client.search(
        search_text=None,
        vector=query_vector,
        top_k=3,
        vector_fields="contentVector",
        select=['source','content'],
)
    results=list(results)
    contents = []
    for result in results:
        content = f'''Source URL: ```{result['source']}```
Content: ```{result['content'].replace('#','').replace('*','')}```'''
        contents.append(content)

    relevant_content = '\n##########\n'.join(contents)
    return relevant_content

In [ ]:
query='i am a working employee, suggest me some aditya birla plans'
relevant_content=get_context(query)

In [ ]:
messages = []
SYSTEM_PROMPT = f"""Role: I am an Expert QnA Bot, mandated to provide answers exclusively from the given Context.
Principle: I provide responses that are strictly derived from Context, void of personal knowledge or insights. Only answers present within Context are permissible.
Objective: Utilize information from multiple sections within the given Context to create a single comprehensive response. If a query falls outside of the Context, the response will be: "Not Found"

The context you will be provided is organized into sections, each with a 'Source URL' and 'Content'. If the answer is found within that specific section, include the corresponding Source URL in your response. If the answer spans over multiple sections, include citations from all relevant parts.
Encoded links in 'Content' are encoded in the format: "actual_text (link)". 
For example: "these types of insurance (https://lifeinsurance.adityabirlacapital.com/life-insurance-basics/different-types-of-life-insurance-plans) are better for the future".

Context:
------
{relevant_content}
------

Output Format: Format your answer as JSON with four keys: 'sources', 'encoded_links', 'recommended_questions' and 'answer'.
The 'sources' key should only contain a list of sources from the 'Source URL' section of Context from where the answer is derived or would be an empty list if the answer is "Not Found".
The 'encoded_links' key should contain a list of unique links that are part of the Context only from where the 'answer' is derived or would be an empty list if there are no urls as part of the answer.
The 'recommended_questions' key should contain a list 3 recommended questions based on the Context and given query that a user can follow up with.
The 'answer' key should be a markdown string formatted according to the below output format instructions. 

Output Format Instructions:
- Use Markdown rendering elements for visually appealing responses.
- Utilize headings when necessary for organizing information into sections.
- Use compact tables for structured data presentation.
- Bold relevant parts of responses for improved readability.
- Use short lists to present multiple items or options concisely.

Mandatory Rules:
- Handle and respond to greeting queries.
- Prohibition: Strictly refrain from responses to queries which does not have the answer in Context.
- Response Language: Responses must be delivered in the English language with consistency.
- Respond in a chatbot-like manner and provide responses according to the user's perspective, using only the given context.
- Include source only from the 'Source URL' section in Context from where the answer was derived. The source can also be PDF names.
- Respond with all the relevant encoded links in 'encoded_links' key.
- The 'recommended_questions' key should only contain 3 questions that have the answers present in the Context. If the 'answer' is "Not Found" the recommended_questions would be an empty list. The questions recommended should not contain the term 'Context'. Always generate 3 recommended questions.
- Answer throughly and completely to the user query. Do not provide any response like "for more details refer Context" in the final answer.
- Never mention that you are answering from "Context" in the final response.
- Important: Each key in the JSON Response should be a standalone/independent one and must not contain any mentions/details to the other keys. The 'answer' key in the JSON Response should not contain the source, recommended questions or encoded links."""
        
messages.append({"role": "system", "content": SYSTEM_PROMPT})
messages.append({"role": "user", "content": f"Query: {query}\nJSON Response: "})

In [ ]:
resp=openai.ChatCompletion.create(messages=messages,engine='gpt-35-turbo',temperature=0, seed=70, max_tokens=1100)
res=resp['choices'][0].message['content']
printmd(get_json_output(res)['answer'])